In [1]:
import sys
print(sys.executable)

c:\Users\rahul\OneDrive\Desktop\Projects\Feature-Store\fst\Scripts\python.exe


In [4]:
import pandas as pd
data = pd.read_csv('diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
predictors_df = data.loc[:,data.columns!='Outcome']
target_df = data['Outcome']

Create timestamp to be added as event_timestamp column in the data set

In [6]:
timestamps = pd.date_range(end = pd.Timestamp.now(),
                           periods = len(data),freq = 'D').to_frame(name = 'event_timestamp', index = False)

In [7]:
timestamps

,event_timestamp
0,2020-11-25 11:42:30.081437
1,2020-11-26 11:42:30.081437
2,2020-11-27 11:42:30.081437
3,2020-11-28 11:42:30.081437
4,2020-11-29 11:42:30.081437
...,...
763,2022-12-28 11:42:30.081437
764,2022-12-29 11:42:30.081437
765,2022-12-30 11:42:30.081437
766,2022-12-31 11:42:30.081437


add event_timestamp column to the predictors and target dataframes

In [8]:
predictors_df = pd.concat(objs = [predictors_df, timestamps], axis = 1)
target_df = pd.concat(objs = [target_df, timestamps], axis =1)

Create a patientID column to uniquely identify records with patientID and timestamp field together.

In [9]:
dataLen = len(data)
idsList = list(range(dataLen))
patient_ids = pd.DataFrame(data = idsList, columns = ['patient_id'])
predictors_df = pd.concat(objs = [predictors_df, patient_ids], axis = 1)
target_df = pd.concat(objs = [target_df, patient_ids], axis =1)

In [10]:
predictors_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,event_timestamp,patient_id
0,6,148,72,35,0,33.6,0.627,50,2020-11-25 11:42:30.081437,0
1,1,85,66,29,0,26.6,0.351,31,2020-11-26 11:42:30.081437,1
2,8,183,64,0,0,23.3,0.672,32,2020-11-27 11:42:30.081437,2
3,1,89,66,23,94,28.1,0.167,21,2020-11-28 11:42:30.081437,3
4,0,137,40,35,168,43.1,2.288,33,2020-11-29 11:42:30.081437,4


In [11]:
predictors_df.to_parquet(path='predictors_df.parquet')
target_df.to_parquet(path='target_df.parquet')

In [12]:
!feast version

Feast SDK Version: "feast 0.21.3"


2. Do feast init

this is option as it creates the feast repo directory structure. you can create a directory using mkdir and inside that create a feature_Store.yaml file and a feature_definitions.py file. but its better to use feast init and then modify the respective files.

In [13]:
!feast init feature_repo


Creating a new Feast repository in c:\Users\rahul\OneDrive\Desktop\Projects\Feature-Store\feature_repo.



C:\Users\rahul\OneDrive\Desktop\Projects\Feature-Store\fst\lib\site-packages\feast\repo_operations.py:312: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.dir_util import copy_tree
C:\Users\rahul\OneDrive\Desktop\Projects\Feature-Store\fst\lib\site-packages\feast\driver_test_data.py:100: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  for dt in pd.date_range(


3. **Update feature store yaml file if needed**
you can update the online store and local store paths in feature_store.yaml file if needed.

4. **Define Feature definitions in a python file inside feature repo directory (created using feast init)**
This step is known as register and deploy the features go inside the feature_repo folder in the github you will see the feature_definition.py file with updated code. modify as per to your dataset features.

5. **Do feast apply**
do feast apply from inside the feature_repo directory**

In [3]:
pwd

'c:\\Users\\rahul\\OneDrive\\Desktop\\Projects\\Feature-Store'

In [4]:
cd feature_repo

c:\Users\rahul\OneDrive\Desktop\Projects\Feature-Store\feature_repo


In [16]:
!feast apply

Created entity patient_id
Created feature view target_df_feature_view
Created feature view predictors_df_feature_view

Created sqlite table feature_repo_predictors_df_feature_view
Created sqlite table feature_repo_target_df_feature_view



C:\Users\rahul\OneDrive\Desktop\Projects\Feature-Store\fst\lib\site-packages\feast\data_source.py:229: DeprecationWarning: The argument 'event_timestamp_column' is being deprecated. Please use 'timestamp_field' instead. instead. Feast 0.23 and onwards will not support the argument 'event_timestamp_column' for datasources.
  warnings.warn(


**6. Generate Training Data Set**

In [8]:
from feast import FeatureStore
import pandas as pd
from feast.infra.offline_stores.file_source import SavedDatasetFileStorage

store = FeatureStore(repo_path='.')

entity_df = pd.read_parquet(path = 'data/target_df.parquet')

training_data = store.get_historical_features(
entity_df = entity_df,
    features = [
        "predictors_df_feature_view:Pregnancies",
        "predictors_df_feature_view:Glucose",
        "predictors_df_feature_view:BloodPressure",
        "predictors_df_feature_view:SkinThickness",
        "predictors_df_feature_view:Insulin",
        "predictors_df_feature_view:BMI",
        "predictors_df_feature_view:DiabetesPedigreeFunction",
        "predictors_df_feature_view:Age",
               ]
)

dataset = store.create_saved_dataset(
from_=training_data,
    name = "diabetes_dataset",
    storage = SavedDatasetFileStorage('data/diabetes_dataset.parquet')
)

c:\Users\rahul\OneDrive\Desktop\Projects\Feature-Store\fst\lib\site-packages\feast\feature_store.py:952: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [9]:
training_data.to_df()

,Outcome,event_timestamp,patient_id,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,1,2020-11-25 11:42:30.081437+00:00,0,6,148,72,35,0,33.6,0.627,50
1,0,2020-11-26 11:42:30.081437+00:00,1,1,85,66,29,0,26.6,0.351,31
2,1,2020-11-27 11:42:30.081437+00:00,2,8,183,64,0,0,23.3,0.672,32
3,0,2020-11-28 11:42:30.081437+00:00,3,1,89,66,23,94,28.1,0.167,21
4,1,2020-11-29 11:42:30.081437+00:00,4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...,...,...,...
763,0,2022-12-28 11:42:30.081437+00:00,763,10,101,76,48,180,32.9,0.171,63
764,0,2022-12-29 11:42:30.081437+00:00,764,2,122,70,27,0,36.8,0.340,27
765,0,2022-12-30 11:42:30.081437+00:00,765,5,121,72,23,112,26.2,0.245,30
766,1,2022-12-31 11:42:30.081437+00:00,766,1,126,60,0,0,30.1,0.349,47


7. Model Training

In [12]:
# Importing dependencies
from feast import FeatureStore
import numpy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from joblib import dump

# Getting our FeatureStore
store = FeatureStore(repo_path=".")

# Retrieving the saved dataset and converting it to a DataFrame
training_df = store.get_saved_dataset(name="diabetes_dataset").to_df()

# Separating the features and labels
y = training_df['Outcome']
X = training_df.drop(
    labels=['Outcome', 'event_timestamp', "patient_id"], 
    axis=1)

# Splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    stratify=y)

# Creating and training LogisticRegression
reg = LogisticRegression(solver='liblinear')
reg.fit(X=X_train[sorted(X_train)], y=y_train)

# Saving the model
dump(value=reg, filename="model.joblib")

c:\Users\rahul\OneDrive\Desktop\Projects\Feature-Store\fst\lib\site-packages\feast\feature_store.py:1005: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(
c:\Users\rahul\OneDrive\Desktop\Projects\Feature-Store\fst\lib\site-packages\sklearn\utils\__init__.py:1067: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)


['model.joblib']

8. **Prepare online feature store**
(Loading the features to online store)

There are two ways you can use to load features to your online store

materialize
materialize loads the latest features between two dates.

feast materialize 2020–01–01T00:00:00 2022–01–01T00:00:00

materialize-incremental
materialize-incremental loads features up to the provided end date (from the last run date):

feast materialize-incremental 2022–01–01T00:00:00

In [13]:
# Importing dependencies
from feast import FeatureStore
from datetime import datetime, timedelta

# Getting our FeatureStore
store = FeatureStore(repo_path=".")

store.materialize_incremental(end_date = datetime.now())

Materializing 2 feature views to 2023-01-01 12:26:38+05:30 into the sqlite online store.

target_df_feature_view from 2022-12-30 06:56:38+05:30 to 2023-01-01 12:26:38+05:30:


100%|████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 259.77it/s]


predictors_df_feature_view from 2022-12-30 06:56:38+05:30 to 2023-01-01 17:56:38+05:30:


100%|████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 351.61it/s]


9. Get online features for prediction

In [14]:
# Importing dependencies
from feast import FeatureStore
import pandas as pd
from joblib import load

# Getting our FeatureStore
store = FeatureStore(repo_path=".")

# Defining our features names
feast_features = [
        "predictors_df_feature_view:Pregnancies",
        "predictors_df_feature_view:Glucose",
        "predictors_df_feature_view:BloodPressure",
        "predictors_df_feature_view:SkinThickness",
        "predictors_df_feature_view:Insulin",
        "predictors_df_feature_view:BMI",
        "predictors_df_feature_view:DiabetesPedigreeFunction",
        "predictors_df_feature_view:Age",
    ]

# Getting the latest features
features = store.get_online_features(
    features=feast_features,    
    entity_rows=[{"patient_id": 767}, {"patient_id": 766}]
).to_dict()

# Converting the features to a DataFrame
features_df = pd.DataFrame.from_dict(data=features)

In [15]:
features_df.head()

,patient_id,Pregnancies,SkinThickness,BMI,Insulin,DiabetesPedigreeFunction,Age,BloodPressure,Glucose
0,767,1,31,30.4,0,0.315,23,70,93
1,766,1,0,30.1,0,0.349,47,60,126


10. Call the predict function and see the output

In [16]:
# Loading our model and doing inference
reg = load("model.joblib")
predictions = reg.predict(features_df[sorted(features_df.drop("patient_id", axis=1))])
print(predictions)

[0 0]


c:\Users\rahul\OneDrive\Desktop\Projects\Feature-Store\fst\lib\site-packages\sklearn\linear_model\_base.py:295: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)
